In [0]:
trigger_location = dbutils.widgets.get("trigger_location")
prompt = dbutils.widgets.get("prompt")
frame_stride = int(dbutils.widgets.get("frame_stride"))
truncate = dbutils.widgets.get("truncate")
threshold = float(dbutils.widgets.get("threshold"))

# trigger_location = '/Volumes/pubsec_video_processing/cv/auto_segment/images/maren_jack.MOV'
# prompt = 'child'
# trigger_location = '/Volumes/pubsec_video_processing/cv/auto_segment/inputs/weimaraner-doc-trimmed.mov'
# prompt = 'weimaraner'
# trigger_location = '/Volumes/pubsec_video_processing/cv/auto_segment/inputs/fresno-traffic-cam-white-box-truck.mov'
# prompt = 'white box truck'
# frame_stride = 30
# truncate = True
# threshold = 0.5

if truncate==0 or truncate=='false' or truncate=='False':
    truncate = False
else:
    truncate=True

print(trigger_location)
print(prompt)
print(frame_stride)
print(truncate)
print(type(truncate))
print(threshold)
print(type(threshold))

/Volumes/pubsec_video_processing/cv/auto_segment/inputs/fresno-traffic-cam-white-box-truck.mov
white box truck
30
True
<class 'bool'>
0.5
<class 'float'>


In [0]:
# dbutils.notebook.exit("Notebook execution stopped by user request.")

In [0]:
from huggingface_hub import login
import os

hf_pat = dbutils.secrets.get("justinm-buildathon-secrets", "hf_pat")
os.environ["HF_TOKEN"] = hf_pat
login(token=hf_pat)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [0]:
import mlflow

model = mlflow.pyfunc.load_model("models:/pubsec_video_processing.cv.transformers-sam3-video@job")

/databricks/python/lib/python3.12/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
Unexpected internal error when monkey patching `Trainer.train`: cannot import name 'HybridCache' from 'transformers' (/local_disk0/.ephemeral_nfs/envs/pythonEnv-839a7f37-a4b7-473d-a16c-35203f2c12bf/lib/python3.12/site-packages/transformers/__init__.py)
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


device: cuda
loading model...
loading processor...
context loaded


In [0]:
import mlflow
import imageio
import numpy as np
import pandas as pd
import cv2
import base64
import timeit
from io import BytesIO
from PIL import Image
import os

In [0]:
from datetime import datetime

if trigger_location.endswith('/') and (trigger_location[-4]!='.' or trigger_location[-5]!='.'):
    # Your volume directory path
    directory_path = trigger_location

    # List all files in the directory
    files = dbutils.fs.ls(directory_path)

    # Filter out directories, keep only files
    files = [f for f in files if not f.isDir()]

    # Sort by modification time (most recent first)
    files_sorted = sorted(files, key=lambda x: x.modificationTime, reverse=True)

    # Get the most recent file
    if files_sorted:
        most_recent_file = files_sorted[0]
        most_recent_path = most_recent_file.path.replace('dbfs:','')
        most_recent_name = most_recent_file.name
        
        print(f"Most recent file: {most_recent_name}")
        print(f"Full path: {most_recent_path}")
        print(f"Modified: {datetime.fromtimestamp(most_recent_file.modificationTime/1000)}")
    else:
        print("No files found in directory")
else:
    most_recent_file = dbutils.fs.ls(trigger_location)[0]
    most_recent_path = most_recent_file.path.replace('dbfs:','')
    most_recent_name = most_recent_file.name

    print(f"Most recent file: {most_recent_name}")
    print(f"Full path: {most_recent_path}")
    print(f"Modified: {datetime.fromtimestamp(most_recent_file.modificationTime/1000)}")

Most recent file: fresno-traffic-cam-white-box-truck.mov
Full path: /Volumes/pubsec_video_processing/cv/auto_segment/inputs/fresno-traffic-cam-white-box-truck.mov
Modified: 2026-01-12 19:27:23


In [0]:
# most_recent_file = trigger_location
# most_recent_name = most_recent_file.split("/")[-1]
# most_recent_path = most_recent_file.replace('dbfs:','')

In [0]:
def write_results(FILE_URL, results, truncate=True, frame_stride=30, speed_up=5):
    import os

    OUTPUT_FILE_URL = FILE_URL.replace("inputs", "outputs")
    output_dir = os.path.dirname(OUTPUT_FILE_URL)
    os.makedirs(output_dir, exist_ok=True)
    FPS = 30

    def decode_mask(encoded_mask: str) -> np.ndarray:
        """Decode base64 mask back to numpy array"""
        buf = BytesIO(base64.b64decode(encoded_mask))
        return np.load(buf)

    def add_text_overlays(frame: np.ndarray, timestamp_sec: float, count_text: str) -> np.ndarray:
        """Add timestamp overlay to frame in the top-right corner"""
        # Convert seconds to HH:MM:SS.ms format
        hours = int(timestamp_sec // 3600)
        minutes = int((timestamp_sec % 3600) // 60)
        seconds = int(timestamp_sec % 60)
        milliseconds = int((timestamp_sec % 1) * 1000)
            
        timestamp_text = f"{hours:02d}:{minutes:02d}:{seconds:02d}.{milliseconds:03d}"
        
        # Create a copy to avoid modifying original
        frame_with_overlay = frame.copy()
        
        # Get frame dimensions
        height, width = frame.shape[:2]
        
        # Set up text properties
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1.5
        font_thickness = 2
        text_color = (255, 255, 255)  # White text
        bg_color = (0, 0, 0)  # Black background
        padding = 10
        
        # Get text size
        (text_width_right, text_height_right), baseline_right = cv2.getTextSize(
            timestamp_text, font, font_scale, font_thickness
        )
        
        # Position in top-right corner
        text_x_right = width - text_width_right - padding
        text_y_right = padding + text_height_right
        
        # Draw semi-transparent background rectangle
        bg_x1_right = text_x_right - 5
        bg_y1_right = text_y_right - text_height_right - 5
        bg_x2_right = text_x_right + text_width_right + 5
        bg_y2_right = text_y_right + baseline_right + 5

        # Get text size
        (text_width_left, text_height_left), baseline_left = cv2.getTextSize(
            count_text, font, font_scale, font_thickness
        )

        # Position in top-left corner
        text_x_left = padding
        text_y_left = padding + text_height_left
        
        # Draw semi-transparent background rectangle
        bg_x1_left = text_x_left - 5
        bg_y1_left = text_y_left - text_height_left - 5
        bg_x2_left = text_x_left + text_width_left + 5
        bg_y2_left = text_y_left + baseline_left + 5

        # Create overlay for semi-transparency
        overlay = frame_with_overlay.copy()
        cv2.rectangle(overlay, (bg_x1_right, bg_y1_right), (bg_x2_right, bg_y2_right), bg_color, -1)
        cv2.rectangle(overlay, (bg_x1_left, bg_y1_left), (bg_x2_left, bg_y2_left), bg_color, -1)
        cv2.addWeighted(overlay, 0.6, frame_with_overlay, 0.4, 0, frame_with_overlay)
        
        # Draw text
        cv2.putText(
            frame_with_overlay,
            timestamp_text,
            (text_x_right, text_y_right),
            font,
            font_scale,
            text_color,
            font_thickness,
            cv2.LINE_AA
        )
        # Draw text
        cv2.putText(
            frame_with_overlay,
            count_text,
            (text_x_left, text_y_left),
            font,
            font_scale,
            text_color,
            font_thickness,
            cv2.LINE_AA
        )
        return frame_with_overlay

    # Detect input video format and setup codec
    import os
    _, input_ext = os.path.splitext(FILE_URL)
    _, output_ext = os.path.splitext(OUTPUT_FILE_URL)
    
    # Use the same extension as input if output doesn't have one
    if not output_ext:
        OUTPUT_FILE_URL = OUTPUT_FILE_URL + input_ext
        output_ext = input_ext
    
    print(f"Input format: {input_ext}, Output format: {output_ext}")
    
    # Open original video to get frames
    print("Processing frames and applying masks...")
    cap = cv2.VideoCapture(FILE_URL)
    fps = cap.get(cv2.CAP_PROP_FPS) or FPS
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"Video properties: {frame_width}x{frame_height} @ {fps} fps")

    # Create a mapping of frame_idx to results
    print(f"Found {len(results)} frames in results")
    result_map = {r["frame_idx"]: r for r in results}
    # print("keys", result_map.keys())

    # Initialize imageio writer for direct frame-by-frame writing with H.264
    import imageio
    import tempfile
    with tempfile.NamedTemporaryFile(suffix=output_ext, delete=False) as tmp_file:
        temp_video_path = tmp_file.name
    
    output_fps = (fps/frame_stride) * speed_up if frame_stride is not None else fps * speed_up
    
    # Use imageio's get_writer for frame-by-frame writing with H.264 (via bundled ffmpeg)
    # This automatically uses imageio-ffmpeg which bundles ffmpeg binaries
    video_writer = imageio.get_writer(
        temp_video_path,
        fps=output_fps,
        codec='libx264',
        pixelformat='yuv420p',
        quality=8  # Quality: 0 (worst) to 10 (best)
    )
    
    print(f"Writing video at {output_fps:.2f} fps using imageio with H.264 codec...")
    
    frame_idx = 0
    saved_images = []
    frames_written = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # Only process frames with segmentation results
        if frame_idx in result_map:
            # print(frame_idx)
            # Convert BGR to RGB
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            res = result_map[frame_idx]

            if res["masks"]:
                # Get all of the masks
                masks = [decode_mask(x) for x in res["masks"]]
                no_of_masks = len(masks)
                
                # Overlay with transparency
                overlay = rgb_frame.copy()
                for mask in masks:
                    overlay[mask > 0.5] = [0, 255, 0]  # Green overlay
                masked_frame = cv2.addWeighted(rgb_frame, 0.7, overlay, 0.3, 0)
                                
            else:
                masked_frame = rgb_frame
                no_of_masks = 0
                        
            # Calculate timestamp for this frame
            timestamp_sec = frame_idx / fps
            
            # Add timestamp overlay (convert back to BGR for cv2 operations, then back to RGB)
            masked_frame_bgr = cv2.cvtColor(masked_frame, cv2.COLOR_RGB2BGR)
            # masked_frame_with_timestamp = add_timestamp(masked_frame_bgr, timestamp_sec)
            # masked_frame_with_timestamp_and_count = add_text_overlay(masked_frame_with_timestamp, f"Count: {len(res['masks'])}")
            masked_frame_with_text_overlays = add_text_overlays(masked_frame_bgr, timestamp_sec, f"Count: {no_of_masks}")
            masked_frame = cv2.cvtColor(masked_frame_with_text_overlays, cv2.COLOR_BGR2RGB)
            
            # Save original frame to saved_images
            if not truncate:
                saved_images.append(Image.fromarray(rgb_frame))
                # Write segmented frame directly to video using imageio (expects RGB)
                video_writer.append_data(masked_frame)
                frames_written += 1
            elif res["masks"]:
                saved_images.append(Image.fromarray(rgb_frame))
                # Write segmented frame directly to video using imageio (expects RGB)
                video_writer.append_data(masked_frame)
                frames_written += 1
            
            # if frames_written % 100 == 0:
            #     print(f"Written {frames_written} frames")
            
        frame_idx += 1

    cap.release()
    video_writer.close()  # Close imageio writer
    
    print(f"Saved {len(saved_images)} frames to memory")
    print(f"Written {frames_written} frames to video with H.264 encoding")

    # 3. Copy video to final destination
    import shutil
    
    temp_size = os.path.getsize(temp_video_path)
    print(f"Video created with H.264: {temp_size:,} bytes ({temp_size/1024/1024:.2f} MB)")

    # Copy to final destination
    print(f"Copying to Volumes: {OUTPUT_FILE_URL}")
    shutil.copy2(temp_video_path, OUTPUT_FILE_URL)

    final_size = os.path.getsize(OUTPUT_FILE_URL)
    print(f"✓ Video successfully saved to: {OUTPUT_FILE_URL}")
    print(f"  Final size: {final_size:,} bytes ({final_size/1024/1024:.2f} MB)")

    # Clean up temporary file
    if os.path.exists(temp_video_path):
        os.remove(temp_video_path)
        print("Cleaned up temporary file")

    return saved_images

In [0]:
def process_file(triggered_file, prompt, resize=True):
    print(f"Triggered by file: {triggered_file}")

    model_input = pd.DataFrame([{
        "video_path": triggered_file,
        "prompt": prompt,
        "frame_stride": frame_stride,  # Process every nth frame
        "batch_size": 32,
        "threshold": threshold,
        "mask_threshold": 0.5
    }])
    results = model.predict(model_input)
    return results

In [0]:
print("Files processed in this run:", most_recent_path)

print("Segmenting video...")
starting_time = timeit.default_timer()
results = process_file(most_recent_path, prompt, resize=False)
print(f"Inference time: {round((timeit.default_timer() - starting_time))} secs")

Files processed in this run: /Volumes/pubsec_video_processing/cv/auto_segment/inputs/fresno-traffic-cam-white-box-truck.mov
Segmenting video...
Triggered by file: /Volumes/pubsec_video_processing/cv/auto_segment/inputs/fresno-traffic-cam-white-box-truck.mov
File to process: /Volumes/pubsec_video_processing/cv/auto_segment/inputs/fresno-traffic-cam-white-box-truck.mov
Video path: /Volumes/pubsec_video_processing/cv/auto_segment/inputs/fresno-traffic-cam-white-box-truck.mov
Prompt: white box truck
Frame stride: 30
Batch size: 32
Threshold: 0.5
Mask threshold: 0.5
Inference time: 858 secs


In [0]:
len(results)

3217

In [0]:
# import pickle

# pickle_path = "/Volumes/pubsec_video_processing/cv/auto_segment/images/results.pkl"
# with open(pickle_path, "wb") as f:
#     pickle.dump(results, f, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# import pickle

# pickle_path = "/Volumes/pubsec_video_processing/cv/auto_segment/images/results.pkl"
# with open(pickle_path, "rb") as f:
#     results = pickle.load(f)

In [0]:
print("Writing segmented video...")
starting_time = timeit.default_timer()
written_images = write_results(most_recent_path, results, truncate=truncate, frame_stride=frame_stride, speed_up=5)
print(f"Writing time: {round((timeit.default_timer() - starting_time))} secs")

Writing segmented video...
Input format: .mov, Output format: .mov
Processing frames and applying masks...
Video properties: 1680x942 @ 59.92300129366106 fps
Found 3217 frames in results
Writing video at 9.99 fps using imageio with H.264 codec...


Written 100 frames
Written 200 frames
Written 300 frames
Written 400 frames
Written 400 frames
Written 500 frames
Written 600 frames
Written 700 frames
Written 800 frames
Written 900 frames
Written 1000 frames
Written 1100 frames
Saved 1158 frames to memory
Written 1158 frames to video with H.264 encoding
Video created with H.264: 337,471,413 bytes (321.84 MB)
Copying to Volumes: /Volumes/pubsec_video_processing/cv/auto_segment/outputs/fresno-traffic-cam-white-box-truck.mov
✓ Video successfully saved to: /Volumes/pubsec_video_processing/cv/auto_segment/outputs/fresno-traffic-cam-white-box-truck.mov
  Final size: 337,471,413 bytes (321.84 MB)
Cleaned up temporary file
Writing time: 249 secs


In [0]:
captioning_model = mlflow.pyfunc.load_model("models:/pubsec_video_processing.cv.transformers-blip@job")

/databricks/python/lib/python3.12/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
Unexpected internal error when monkey patching `Trainer.train`: cannot import name 'HybridCache' from 'transformers' (/local_disk0/.ephemeral_nfs/envs/pythonEnv-4e29edd4-f093-45cf-bad3-4798f385244f/lib/python3.12/site-packages/transformers/__init__.py)


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

The image processor of type `BlipImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/616 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie text_decoder.bert.embeddings.word_embeddings.weight to text_decoder.cls.predictions.decoder.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
BlipForConditionalGeneration LOAD REPORT from: Salesforce/blip-image-captioning-large
Key                                       | Status     |  | 
------------------------------------------+------------+--+-
text_decoder.bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [0]:
from PIL import Image
from io import BytesIO
import base64

def pil_to_base64_str(img: Image.Image, format: str = "PNG") -> str:
    """
    Convert a PIL image to a Base64-encoded string.
    
    Args:
        img: PIL.Image.Image
        format: image format, e.g., 'PNG' or 'JPEG'
        
    Returns:
        str: Base64 string that can be safely passed in JSON
    """
    buf = BytesIO()
    img.save(buf, format=format)
    buf.seek(0)
    b64_str = base64.b64encode(buf.read()).decode("utf-8")
    return b64_str

In [0]:
from io import BytesIO

written_images_str = [pil_to_base64_str(x) for x in written_images]

In [0]:
model_input = {
    "image_path": written_images_str
}

captions = captioning_model.predict(model_input)

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import ChatMessage, ChatMessageRole

# Initialize the client
w = WorkspaceClient()

# Define your endpoint name
endpoint_name = "databricks-gpt-5-2"

# # Create a chat message
# messages = [
#     ChatMessage(
#         role=ChatMessageRole.USER,
#         content=f"The following is a list of descriptions of images. The images represent frames from a video, in order. Can you identify anything out of the ordinary or anomalous in the video? When I say anomlous, I mean actions taking place that are against the law or violent in nature. If you find something that shouldn't be happening, describe why it is anomolous. If you don't find anything abnormal, just respond with 'Nothing anomolous found.' \n\n {captions}"
#     )
# ]
# Create a chat message
captions = list(set(captions))
prompt = f"""Summarize the following sections of a video with a focus on what is functionally happening over time. Be descriptive. The resulting summary should be a functional narrative. Return this as Markdown text - and only markdown. \n\n {captions}"""

# messages = [
#     ChatMessage(
#         role=ChatMessageRole.USER,
#         content=f"The following is a list of descriptions of images. The images represent frames from a video, in order. There may be obvious scene changes based on the descriptions. Please summarize the contents of each scene in the video in a single sentence each.' \n\n {captions}"
#     )
# ]

messages = [
    ChatMessage(
        role=ChatMessageRole.USER,
        content=prompt
    )
]

# Query the endpoint
response = w.serving_endpoints.query(
    name=endpoint_name,
    messages=messages,
    # max_tokens=500  # optional parameter
)

# Access the response
r = response.choices[0].message.content
# print(r)
try:
    text = [x for x in r if x['type'] == 'text'][0]['text']
except:
    text = r
print(text)

## Functional Narrative Summary (Over Time)

The video alternates between two main settings: an organized **dog show** and more casual **outdoor play by fields and water**. It moves back and forth, but the overall progression is from **handling and evaluation** to **active retrieving and water play**, with recurring cutaways to other animals near the same waterside environment.

### 1) Dog show handling and judging
A woman—often in a **red dress**—is shown repeatedly **petting and presenting dogs on leashes** in a show environment. Dogs are walked in a **ring** or along a **stage**, guided carefully to display posture and movement. At moments, the woman and a man in a **tan/beige suit** perform more formal checks: the dog is **examined up close**, including a sequence where a **stethoscope is used** and another where the dog’s **teeth/mouth are inspected**, consistent with judging or veterinary-style evaluation. Grooming appears as part of preparation: a dog is **brushed/handled** whil

In [0]:
txt_filename = most_recent_path.split('.')[0] + '.txt'
txt_filename = txt_filename.replace('/inputs/', '/descriptions/')
print(txt_filename)

dbutils.fs.put(txt_filename, text, True)

/Volumes/pubsec_video_processing/cv/auto_segment/descriptions/weimaraner-doc-trimmed.txt
Wrote 4316 bytes.


True